In [4]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [6]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [8]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [3]:
# Load the LLAMA2 model and tokenizer
model_name = "NousResearch/llama-2-7b-chat-hf"  # Replace with the path to your LLAMA2 model
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

In [ ]:


# Define the prompt
prompt = """
System: You are a SQL generation assistant.
User: Please generate an SQL query to retrieve all orders placed in the last month.
User: The database schema includes tables 'orders' and 'order_date.'
"""

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True)

# Generate an SQL query
output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode and print the SQL query
sql_query = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated SQL Query:", sql_query)